In [1]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_colwidth', 50)

In [3]:
colnames = ['id', 'workName', 'overallRating', 'review']
df = pd.read_csv(os.path.abspath('Datasets\\animeReviewsOrderByTime.csv'), usecols=np.arange(15))
df = df[colnames]
df = df.dropna()
df = df.drop(df[df['review'] == ''].index)
df.head(10)

C:\Users\Armand Syahtama\envs\csi4106\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,workName,overallRating,review
0,8121,Cowboy_Bebop,10,Cowboy Bebop is an episodic series. By episodi...
1,63480,Utawarerumono,8,Utawarerumono manages to be one of those harem...
2,8452,Hajime_no_Ippo,10,"First, let me say that I\\'m not a fan of boxi..."
3,66544,Gensoumaden_Saiyuuki,9,Saiyuki is one of those animes that just grabs...
4,55936,Ranma_½,7,A comedy/romance based on the manga by Rumiko ...
5,22039,Kino_no_Tabi__The_Beautiful_World,9,What do you say about an anime that is just ab...
6,68626,Kareshi_Kanojo_no_Jijou,8,Kare Kano was a romance anime that could have ...
7,18797,Hunter_x_Hunter,10,Overall: My very best anime and i haven't act...
8,43899,Golden_Boy,10,Overall:Honestly I really don't care about oth...
9,18796,Hunter_x_Hunter,10,Hunters… What do you think about when you hear...


In [4]:
import string
import wordninja
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer

english_stopwords = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
detokenizer = TreebankWordDetokenizer()
punctuation_to_remove = string.punctuation
punctuation_to_remove = punctuation_to_remove.replace('.', '')

def preprocess_text(text: str, split_word: bool = True):
    text = text.lower() # lower the case
    text = text.strip()
    text = "".join([c if c not in punctuation_to_remove else ' ' for c in text]) # take out punctuation
    tokens = word_tokenize(text) # tokenize text
    tokens = [w for w in tokens if w not in english_stopwords] # remove stopwords
    tokens = [lemmatizer.lemmatize(w) for w in tokens] # lemmatize all the words
    tokens = [w for w in tokens if w.isalpha()]
    return detokenizer.detokenize(tokens)

In [5]:
df['review'] = df['review'].apply(lambda x: preprocess_text(x))
df['review'].head(10)

0    cowboy bebop episodic series episodic mean one...
1    utawarerumono manages one harem anime anyone p...
2    first let say fan boxing fact pretty much hate...
3    saiyuki one anime grab first episode let go ev...
4    comedy romance based manga rumiko takahashi ra...
5    say anime traveler journeying different countr...
6    kare kano romance anime could become incredibl...
7    overall best anime actually seen anything else...
8    overall honestly really care others opinion an...
9    think hear anime people killing poor cute anim...
Name: review, dtype: object

In [6]:
# determines whether score is positive (1) or not (0)
def determine_sentiment(score: int):
    return 1 if score > 5 else 0

df['sentiment'] = df['overallRating'].apply(lambda x: determine_sentiment(x))
df.head(10)

,id,workName,overallRating,review,sentiment
0,8121,Cowboy_Bebop,10,cowboy bebop episodic series episodic mean one...,1
1,63480,Utawarerumono,8,utawarerumono manages one harem anime anyone p...,1
2,8452,Hajime_no_Ippo,10,first let say fan boxing fact pretty much hate...,1
3,66544,Gensoumaden_Saiyuuki,9,saiyuki one anime grab first episode let go ev...,1
4,55936,Ranma_½,7,comedy romance based manga rumiko takahashi ra...,1
5,22039,Kino_no_Tabi__The_Beautiful_World,9,say anime traveler journeying different countr...,1
6,68626,Kareshi_Kanojo_no_Jijou,8,kare kano romance anime could become incredibl...,1
7,18797,Hunter_x_Hunter,10,overall best anime actually seen anything else...,1
8,43899,Golden_Boy,10,overall honestly really care others opinion an...,1
9,18796,Hunter_x_Hunter,10,think hear anime people killing poor cute anim...,1


In [7]:
df['sentiment'].value_counts()

1    109376
0     25823
Name: sentiment, dtype: int64

In [8]:
df['review'].replace('', np.nan, inplace=True)
df.dropna(subset=['review'], inplace=True)
df.to_csv('Datasets\\processedAnimeReviews.csv', encoding='utf-8', index=False)